In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2018-09-18 19:00:47.085726


In [2]:
secret = False #Will be put to True for a CASD export

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }<\style>"))

In [4]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle

In [5]:
pickle_year_variable_path = "C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/year_variable/"
year =  pickle.load(open(pickle_year_variable_path+"year_variable.p",'rb')); print(year)

2014


In [6]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5'
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/reaction.h5'
pickles_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/pickles/{}'.format(year)
Store = pd.HDFStore(store_path)

In [7]:
for key in Store.keys(): print(key)
Store.close()

/DECES
/DESCENDANCE
/EAR2004_FAMILLE
/EAR2004_INDIVIDU
/EAR2004_LOGEMENT
/EAR2005_FAMILLE
/EAR2005_INDIVIDU
/EAR2005_LOGEMENT
/EAR2006_FAMILLE
/EAR2006_INDIVIDU
/EAR2006_LOGEMENT
/EAR2007_FAMILLE
/EAR2007_INDIVIDU
/EAR2007_LOGEMENT
/EAR2008_FAMILLE
/EAR2008_INDIVIDU
/EAR2008_LOGEMENT
/EAR2009_FAMILLE
/EAR2009_INDIVIDU
/EAR2009_LOGEMENT
/EAR2010_FAMILLE
/EAR2010_INDIVIDU
/EAR2010_LOGEMENT
/EAR2011_FAMILLE
/EAR2011_INDIVIDU
/EAR2011_LOGEMENT
/EAR2012_FAMILLE
/EAR2012_INDIVIDU
/EAR2012_LOGEMENT
/EAR2013_FAMILLE
/EAR2013_INDIVIDU
/EAR2013_LOGEMENT
/EAR2014_FAMILLE
/EAR2014_INDIVIDU
/EAR2014_LOGEMENT
/EAR2015_FAMILLE
/EAR2015_INDIVIDU
/EAR2015_LOGEMENT
/EAR2016_FAMILLE
/EAR2016_INDIVIDU
/EAR2016_LOGEMENT
/FE
/FISC_INDIVIDU_2011
/FISC_INDIVIDU_2012
/FISC_INDIVIDU_2013
/FISC_INDIVIDU_2014
/FISC_INDIVIDU_2015
/FISC_INDIVIDU_2016
/FISC_LOGEMENT_2011
/FISC_LOGEMENT_2012
/FISC_LOGEMENT_2013
/FISC_LOGEMENT_2014
/FISC_LOGEMENT_2015
/FISC_LOGEMENT_2016
/FISC_REVDET_2011
/FISC_REVDET_2012
/FISC_REVDE

In [8]:
fisc_i = pd.read_hdf( store_path,"FISC_INDIVIDU_{}".format(year+1) )
fisc_revdet = pd.read_hdf( store_path,"FISC_REVDET_{}".format(year+1) )

In [9]:
fisc_revenu = pd.read_hdf(store_path,"FISC_REVENU_{}".format(year+1))

In [10]:
fisc_individu = pd.read_hdf( store_path,"FISC_INDIVIDU_{}".format(year+1) )

In [11]:
fisc_revdet.TYPE_FISC = fisc_revdet.TYPE_FISC.astype("str")

In [12]:
df_merge_fisci_revdet = pd.merge(fisc_individu, fisc_revdet,
                              on = ["ID_FISC_FOY_DIFF","ORDREFIP","TYPE_FISC"]).sort_values("ID_FISC_LOG_DIFF")

In [13]:
fisc_revdet.columns[-7:]

Index(['YSALI', 'YCHOI', 'YRSTI', 'YALRI', 'YRAGI', 'YBICI', 'YBNCI'], dtype='object')

In [14]:
result_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'
df_fisci_child_parent = pd.read_hdf(result_store_path, "fisci_child_or_parent_{}".format(year))
df_fisci_child_parent = df_fisci_child_parent.rename(columns={"ID_FISC_FOY_DIFF_y":"ID_FISC_FOY_DIFF"})

In [15]:
## Add parents or children attributes (to determine if TYPEFISC == 1 are children detached or parents)
df_merge_fisci_revdet2 = pd.merge(df_merge_fisci_revdet, df_fisci_child_parent, on = ["ID_FISC_FOY_DIFF","ORDREFIP","TYPE_FISC"], suffixes=("", "_y"), how = "left")

In [16]:
df = df_merge_fisci_revdet2

In [17]:
df.Child.value_counts(dropna = False)

NaN      4486263
False     731735
True      200389
Name: Child, dtype: int64

In [18]:
## Salaire_père Salaire_mère

In [19]:
income_var = ['YSALI', 'YCHOI', 'YRSTI', 'YALRI', 'YRAGI', 'YBICI', 'YBNCI']

In [20]:
%%time
for var in income_var:    
    df["{}_pere".format(var)] = df[(df.SEXE == 1) & (~(df.Child==True))][var]  #condition father and not child
    df["{}_mere".format(var)] = df[(df.SEXE == 2) & (~(df.Child==True))][var] #condition child
    df["{}_children".format(var)] = df[(df.Child == True)].groupby("ID_FISC_LOG_DIFF").sum()[var] #sum all children income

Wall time: 1min 33s


In [21]:
income_var_all = income_var + [var + "_mere" for var in income_var] + [var + "_pere" for var in income_var]

In [22]:
df_income_by_log = df[income_var_all+ ["ID_FISC_LOG_DIFF"]].groupby("ID_FISC_LOG_DIFF").sum()

In [23]:
fisc_revenu2 = pd.merge(fisc_revenu.reset_index(), df_income_by_log.reset_index(), on = "ID_FISC_LOG_DIFF", how = "left" )

In [24]:
## Age père age mère

In [25]:
df = df_merge_fisci_revdet2

In [26]:
df["Age_pere"] =year - df[(df.SEXE == 1) & (~(df.Child==True))].ANAIS
df["Age_mere"] = year - df[(df.SEXE == 2) & (~(df.Child==True))].ANAIS
df["ANAIS_pere"] = df[(df.SEXE == 1) & (~(df.Child==True))].ANAIS
df["ANAIS_mere"] = df[(df.SEXE == 2) & (~(df.Child==True))].ANAIS

In [27]:
df["ID_DIFF_pere"] = df[(df.SEXE == 1) & (~(df.Child==True))].ID_DIFF
df["ID_DIFF_mere"] = df[(df.SEXE == 2) & (~(df.Child==True))].ID_DIFF

In [28]:
df.Age_pere.max()

114.0

In [29]:
df.Parent

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
23         NaN
24         NaN
25         NaN
26         NaN
27         NaN
28         NaN
29         NaN
          ... 
5418357    NaN
5418358    NaN
5418359    NaN
5418360    NaN
5418361    NaN
5418362    NaN
5418363    NaN
5418364    NaN
5418365    NaN
5418366    NaN
5418367    NaN
5418368    NaN
5418369    NaN
5418370    NaN
5418371    NaN
5418372    NaN
5418373    NaN
5418374    NaN
5418375    NaN
5418376    NaN
5418377    NaN
5418378    NaN
5418379    NaN
5418380    NaN
5418381    NaN
5418382    NaN
5418383    NaN
5418384    NaN
5418385    NaN
5418386    NaN
Name: Parent, Length: 5418387, dtype: object

In [30]:
df_age_by_log_count = df[["Age_pere", "Age_mere",
                    "ANAIS_pere", "ANAIS_mere",
                    "ID_FISC_LOG_DIFF"]].groupby("ID_FISC_LOG_DIFF").count()

In [31]:
double_pere_id_log = df_age_by_log_count[df_age_by_log_count.Age_pere == 2].index

In [ ]:
fisc_individu[fisc_individu.ID_FISC_LOG_DIFF.isin(double_pere_id_log)]

In [33]:
df_age_by_log = df[["Age_pere", "Age_mere",
                    "ANAIS_pere", "ANAIS_mere",
                    "ID_FISC_LOG_DIFF"]].groupby("ID_FISC_LOG_DIFF").sum()

In [34]:
fisc_revenu3 = pd.merge(fisc_revenu2.reset_index(), df_age_by_log.reset_index(), on = "ID_FISC_LOG_DIFF", how = "left" )

In [35]:
### Add logement informations

In [36]:
df_fisc_logement = pd.read_hdf( store_path,"FISC_LOGEMENT_{}".format(year+1))

In [37]:
dict(on = "ID_FISC_LOG_DIFF", how = "left" )

{'how': 'left', 'on': 'ID_FISC_LOG_DIFF'}

In [38]:
%time
merge4 = pd.merge(fisc_revenu3, df_fisc_logement, **dict(on = "ID_FISC_LOG_DIFF", how = "left" ))

Wall time: 0 ns


In [39]:
merge4.CHAUFF.value_counts(normalize=True)

O    0.832417
N    0.167583
Name: CHAUFF, dtype: float64

In [49]:
result_store_path

'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'

In [41]:
merge4.Age_pere.max()

542.0

In [42]:
print(year)

2014


In [69]:
result_store_path_csv = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/result/csv/'

In [ ]:
merge4.to_csv(result_store_path_csv+ "step_3_{}.csv".format(year))

In [ ]:
merge4.to_csv(result_store_path_csv)

In [48]:
year

2014

In [50]:
result_store_path

'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'

In [45]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)

now:  2018-09-18 19:06:01.343156
execution_time:  0:05:14.257430


In [ ]:
merge4